In [ ]:
%%html
<style>
.container {
  width: 100%;
}
</style>

In [ ]:
%load_ext nb_mypy

In [ ]:
import heapq
import nbimporter
from AIBaseClass import ChessAI
from Exercise03AI import Exercise03AI
from Exercise04AI import Exercise04AI

# Experimentelle KI

Diese KI-Version implementiert mehrere Ideen für eine erhöhte Performance:

1. Rotierender Cache der letzten zwei Halbzüge statt vollständiges Caching
2. Kein Progressive Deepening aber weiterhin Zugsortierung mithilfe des Cache
3. Entfernen der Tiefe aus dem Cache-Key für bessere Zugsortierung und mehr Cache-Hits

Aus Zeitgründen ist diese KI nur rudimentär beschrieben und besitzt keine eigenen Unit-Tests. Die folgenden Statistiken zeigen eine deutliche Steigerung der Berechnungsgeschwindigkeit, bei weiterhin gleichbleibenden Ergebnissen. Demnach ist diese Version die bisher schnellste KI.

Die folgende Statistik zeigt, dass die `ExperimentalAI` ähnliche Ergebnisse gegen Stockfish erzielt wie die vorherige Version `Exercise08AI`.
<img src="images/Stockfish_vs_ExperimentalAI.png" width="1000px" />

Das folgende Bild zeigt, dass die `ExperimentalAI` durchschnittlich am Schnellsten rechnet pro Zug rechnet.
<img src="images/ExperimentalAI_duration.png" width="1200px" />

Die folgende Statistik zeigt, dass die `ExperimentalAI` durchschnittlich die geringste Anzahl an Knoten untersucht.
<img src="images/ExperimentalAI_nodes.png" width="1200px" />

Das folgende Bild zeigt, dass die `ExperimentalAI` eine geringere Cache-Größe aufweist, als die vorherige Version `Exercise08AI`.
<img src="images/ExperimentalAI_cache_size.png" width="1200px" />

Die folgende Statistik zeigt, dass die `ExperimentalAI` in etwa gleich viele Cache Hits wie die vorherige Version `Exercise08AI` aufweist.
<img src="images/ExperimentalAI_cache_hits.png" width="1200px" />

In [ ]:
import chess
from typing import Any, Callable


class ExperimentalAI(Exercise03AI):
    """Chooses middle game moves using minimax algorithm, alpha-beta-pruning,
    memoization, progressive deepening and the singular value extension."""

    def __init__(self, max_depth: int = 8, **kwargs) -> None:
        super().__init__(**kwargs)
        self.cache_new: dict[tuple, Any] = {}
        self.cache_old: dict[tuple, Any] = {}
        self.MAX_SVE_DEPTH: int = max(max_depth, kwargs["search_depth"])

    def reset(self) -> None:
        """Resets all internal variables."""
        super().reset()
        self.cache_new.clear() # NEW: current move calculation cache
        self.cache_old.clear() # NEW: last move calculation cache

In [ ]:
class ExperimentalAI(ExperimentalAI):  # type: ignore
    def is_quiet_move_and_push(self, board: chess.Board, move: chess.Move) -> bool:
        """Checks if the next move was an promotion, capture or check move. Pushes the given move on the board."""
        if move.promotion or board.piece_type_at(move.to_square):
            board.push(move)
            return False
        board.push(move)
        if board.is_check():
            return False
        return True

In [ ]:
class ExperimentalAI(ExperimentalAI):  # type: ignore
    def minimax_early_abort(
        self, board: chess.Board, level: int, depth: int, current_evaluation: int
    ) -> int | None:
        """Returns an evaluation iff the minimax has an early exit condition. Returns None otherwise."""
        is_checkmate = board.is_checkmate()
        if is_checkmate and not board.turn:
            # White has won the game
            evaluation = self.LIMIT - level
            return evaluation

        if is_checkmate and board.turn:
            # Black has won the game
            evaluation = -self.LIMIT + level
            return evaluation

        if (
            board.is_insufficient_material()
            or not board.legal_moves
            or board.is_fifty_moves()
        ):
            # Game is a draw
            return 0

        # Recursion abort case
        if depth == 0:
            return current_evaluation

        return None

In [ ]:
class ExperimentalAI(ExperimentalAI):  # type: ignore
    def maxValue(
        self,
        board: chess.Board,
        current_evaluation: int,
        depth: int,
        level: int,
        alpha: int,
        beta: int,
    ) -> tuple[int, chess.Move | None]:
        """Searches the best value with given depth using minimax algorithm."""
        self.stats[-1]["nodes"] += 1
        early_abort_evaluation = self.minimax_early_abort(board, level, depth, current_evaluation)
        if early_abort_evaluation is not None:
            self.stats[-1]["leaf_ctr"] += 1
            self.stats[-1]["depth_sum"] += level  # for calculating the average depth
            return early_abort_evaluation, None

        best_move = None
        moves: list[tuple[int, int, chess.Move]] = []

        # White to play (positive numbers are good)
        if level < self.DEPTH - 2: # NEW: only sort moves if there should be something in the cache
            for i, move in enumerate(board.legal_moves):
                board.push(move)
                key = self.get_key(board) # NEW: depth is not relevant here now
                board.pop()
                old_eval = self.cache_old.get(key, (None, i, None))[1]
                heapq.heappush(moves, (-old_eval, i, move))
        else:
            moves = list(enumerate(board.legal_moves))
        maxEvaluation = alpha
        while moves:
            move = heapq.heappop(moves)[-1]
            new_evaluation = self.incremental_evaluate(board, current_evaluation, move)
            if depth > 1 or level + 1 == self.MAX_SVE_DEPTH:
                new_depth = depth
                board.push(move)
            elif self.is_quiet_move_and_push(board, move):
                new_depth = depth
            else:
                new_depth = depth + 1
            evaluation, _, = self.minimax(
                self.minValue,
                board,
                new_evaluation,
                new_depth - 1,
                level + 1,
                maxEvaluation,
                beta,
            )
            board.pop()
            if evaluation >= beta:
                return evaluation, move
            if evaluation > maxEvaluation:
                best_move = move
                maxEvaluation = evaluation
        return maxEvaluation, best_move

In [ ]:
class ExperimentalAI(ExperimentalAI):  # type: ignore
    def minValue(
        self,
        board: chess.Board,
        current_evaluation: int,
        depth: int,
        level: int,
        alpha: int,
        beta: int,
    ) -> tuple[int, chess.Move | None]:
        """Searches the best value with given depth using minimax algorithm."""
        self.stats[-1]["nodes"] += 1
        early_abort_evaluation = self.minimax_early_abort(
            board, level, depth, current_evaluation
        )
        if early_abort_evaluation is not None:
            self.stats[-1]["leaf_ctr"] += 1
            self.stats[-1]["depth_sum"] += level
            return early_abort_evaluation, None

        best_move = None
        moves: list[tuple[int, int, chess.Move]] = []

        # Black to play (negative numbers are good)
        if level < self.DEPTH - 2: # NEW: only sort moves if there should be something in the cache
            for i, move in enumerate(board.legal_moves):
                board.push(move)
                key = self.get_key(board) # NEW: depth is not relevant here now
                board.pop()
                old_eval = self.cache_old.get(key, (None, i, None))[1]
                heapq.heappush(moves, (old_eval, i, move))
        else:
            moves = list(enumerate(board.legal_moves))
        minEvaluation = beta
        while moves:
            move = heapq.heappop(moves)[-1]
            new_evaluation = self.incremental_evaluate(board, current_evaluation, move)
            if depth > 1 or level + 1 == self.MAX_SVE_DEPTH:
                new_depth = depth
                board.push(move)
            elif self.is_quiet_move_and_push(board, move):
                new_depth = depth
            else:
                new_depth = depth + 1
            evaluation, _ = self.minimax(
                self.maxValue,
                board,
                new_evaluation,
                new_depth - 1,
                level + 1,
                alpha,
                minEvaluation,
            )
            board.pop()
            if evaluation <= alpha:
                return evaluation, move
            if evaluation < minEvaluation:
                best_move = move
                minEvaluation = evaluation
        return minEvaluation, best_move

In [ ]:
class ExperimentalAI(ExperimentalAI):  # type: ignore
    def get_key(self, board: chess.Board) -> tuple:
        """Calculates a key that uniquely identifies a given board."""
        return (
            board.pawns,
            board.knights,
            board.bishops,
            board.rooks,
            board.queens,
            board.kings,
            board.occupied_co[chess.WHITE],
            board.occupied_co[chess.BLACK],
            board.turn,
            board.castling_rights,
            board.halfmove_clock
            if board.halfmove_clock >= (50 - self.MAX_SVE_DEPTH)
            else -42,
        ) # NEW: no depth in the key anymore

    def store_in_cache(
        self, key: tuple, result: tuple, depth: int, alpha: int, beta: int
    ) -> None:
        """Stores the result of a minimax computation in the cache."""
        evaluation, move = result
        if evaluation <= alpha:
            self.cache_new[key] = ("≤", evaluation, move, depth) # NEW: store depth as value inside the cache
        elif evaluation < beta:
            self.cache_new[key] = ("=", evaluation, move, depth)
        else:
            self.cache_new[key] = ("≥", evaluation, move, depth)

    def get_from_cache(
        self,
        minValue_or_maxValue: Callable,
        key: tuple,
        board: chess.Board,
        current_eval: int,
        depth: int,
        level: int,
        alpha: int,
        beta: int,
    ) -> tuple:
        """Gets a result from the cache if possible."""
        flag, evaluation, move, eval_depth = self.cache_new.get(
            key
        ) or self.cache_old.get(key)
        if depth > eval_depth: # NEW: if depth isn't sufficient do a normal search
            result = minValue_or_maxValue(
                board, current_eval, depth, level, alpha, beta
            )
            self.store_in_cache(key, result, depth, alpha, beta)
            return result
        if flag == "=":
            return evaluation, move
        elif flag == "≤":
            if evaluation <= alpha:
                return evaluation, move
            elif evaluation < beta:
                result = minValue_or_maxValue(
                    board, current_eval, depth, level, alpha, evaluation
                )
                self.store_in_cache(key, result, depth, alpha, evaluation)
                return result
            else:
                result = minValue_or_maxValue(
                    board, current_eval, depth, level, alpha, beta
                )
                self.store_in_cache(key, result, depth, alpha, beta)
                return result
        else:
            if evaluation <= alpha:
                result = minValue_or_maxValue(
                    board, current_eval, depth, level, alpha, beta
                )
                self.store_in_cache(key, result, depth, alpha, beta)
                return result
            elif evaluation < beta:
                result = minValue_or_maxValue(
                    board, current_eval, depth, level, evaluation, beta
                )
                self.store_in_cache(key, result, depth, evaluation, beta)
                return result
            else:
                return evaluation, move

In [ ]:
class ExperimentalAI(ExperimentalAI):  # type: ignore
    def minimax(
        self,
        minValue_or_maxValue: Callable,
        board: chess.Board,
        current_evaluation: int,
        depth: int,
        level: int = 0,
        alpha: int = -ExperimentalAI.LIMIT,
        beta: int = ExperimentalAI.LIMIT,
    ) -> tuple[int, chess.Move | None]:
        """Searches the best value with given depth using minimax algorithm."""
        key = self.get_key(board)
        self.stats[-1]["cache_tries"] += 1
        self.stats[-1]["max_depth"] = max(level, self.stats[-1]["max_depth"])

        if key in self.cache_new or key in self.cache_old:
            self.stats[-1]["cache_hits"] += 1
            return self.get_from_cache(
                minValue_or_maxValue,
                key,
                board,
                current_evaluation,
                depth,
                level,
                alpha,
                beta,
            )
        result = minValue_or_maxValue(
            board, current_evaluation, depth, level, alpha, beta
        )
        self.store_in_cache(key, result, depth, alpha, beta)
        return result

In [ ]:
import sys
from datetime import datetime


class ExperimentalAI(ExperimentalAI):  # type: ignore
    def get_next_middle_game_move(self, board: chess.Board) -> chess.Move:
        """Gets the best next move."""
        self.last_evaluation: int | None  # type annotation for mypy
        self.stats[-1]["cache_hits"] = 0
        self.stats[-1]["cache_tries"] = 0
        self.stats[-1]["leaf_ctr"] = 0
        self.stats[-1]["max_depth"] = -1
        self.stats[-1]["avg_depth"] = -1
        self.stats[-1]["depth_sum"] = 0

        if self.is_king_endgame != self.check_king_endgame(board):
            self.last_evaluation += self.get_endgame_evaluation_change(board)
        # Calculate current evaluation
        if self.last_evaluation is None:  # type: ignore
            current_evaluation = self.full_evaluate(board)
        else:
            # Get current evaluation (after opponent move)
            last_move = board.pop()
            current_evaluation = self.incremental_evaluate(board, self.last_evaluation, last_move)  # type: ignore
            board.push(last_move)

        evaluation_function = self.maxValue if board.turn else self.minValue
        self.stats[-1]["nodes"] = 0
        future_evaluation, best_move = self.minimax(
            evaluation_function, board, current_evaluation, self.DEPTH
        ) # NEW: no iterative deepening

        # Debugging fail safe
        assert best_move, f"""
        Best move is None with fen '{board.fen()}' at player {type(self).__name__}! 
        depth: {self.DEPTH}, last_eval: {self.last_evaluation}, current_evaluation: {current_evaluation},
        is_king_engame: {getattr(self, 'is_king_endgame', "N/A")}, move_stack: {board.move_stack}
        """
        # Update last evaluation (after player move)
        self.last_evaluation = self.incremental_evaluate(
            board, current_evaluation, best_move
        )
        # Update stats
        self.stats[-1]["minimax_eval"] = future_evaluation
        self.stats[-1]["board_eval_before_move"] = current_evaluation
        self.stats[-1]["board_eval_after_move"] = self.last_evaluation
        self.stats[-1]["avg_depth"] = self.stats[-1]["depth_sum"] / (
            self.stats[-1]["leaf_ctr"] or 1
        )
        del self.stats[-1]["depth_sum"]
        del self.stats[-1]["leaf_ctr"]
        self.stats[-1]["cache_size_mb"] = round(
            (sys.getsizeof(self.cache_new) + sys.getsizeof(self.cache_old))
            / (1024 * 1024),
            2,
        )
        self.cache_old.clear() # NEW: rotate cache
        self.cache_old = self.cache_new
        self.cache_new = {}

        self.stats[-1]["datetime"] = datetime.now().strftime("%H:%M:%S")
        return best_move

## Debugging Bereich

Die folgenden Zellen enthalten Unit-Tests der oben implementierten Funktionen.

In [ ]:
import Exercise02AI as Exercise02AI_
import Exercise04AI as Exercise04AI_

In [ ]:
# Create player and board
unit_test_player = ExperimentalAI(player_name="ExpAI", search_depth=3, max_depth=3)
board = chess.Board("5rk1/1b3p2/8/3p4/3p2P1/2Q4B/5P1K/R3R3 b - - 0 36")
board

In [ ]:
# Test minimax
def test_minimax(
    unit_test_player: ChessAI,
    board: chess.Board,
    current_evaluation: int,
    expected_evaluation: int,
    expected_move: str,
    expected_nodes: int,
    expected_cache_tries: int,
    expected_cache_hits: int,
    expected_cache_elements: int,
):
    unit_test_player.cache_new = {}  # Clear cache
    unit_test_player.cache_old = {}  # Clear cache
    unit_test_player.stats[-1]["cache_tries"] = 0
    unit_test_player.stats[-1]["cache_hits"] = 0
    unit_test_player.stats[-1]["nodes"] = 0
    unit_test_player.stats[-1]["leaf_ctr"] = 0
    unit_test_player.stats[-1]["max_depth"] = 0
    unit_test_player.stats[-1]["depth_sum"] = 0
    f = unit_test_player.maxValue if board.turn else unit_test_player.minValue
    mm_evaluation, mm_move = unit_test_player.minimax(
        f, board, current_evaluation, unit_test_player.DEPTH
    )
    nodes = unit_test_player.stats[-1]["nodes"]
    cache_tries = unit_test_player.stats[-1]['cache_tries']
    cache_hits = unit_test_player.stats[-1]['cache_hits']
    print(f"Minimax Evaluation: {mm_evaluation}")
    print(f"Minimax Move: {mm_move}")
    print(f"Nodes searched: {nodes}")
    print(f"Cache tries: {cache_tries}")
    print(f"Cache hits: {cache_hits}")
    print(f"Elements in new cache: {len(unit_test_player.cache_new)}")
    print(f"Elements in old cache: {len(unit_test_player.cache_old)}")
    assert (
        mm_evaluation == expected_evaluation
    ), "Minimax evaluation does not match expected value!"
    assert mm_move.uci() == expected_move, "Minimax move does not match expected value!"
    assert nodes == expected_nodes, "Searched node count has changed!"
    assert cache_tries == expected_cache_tries, "Cache tries do not match expected value!"
    assert cache_hits == expected_cache_hits, "Cache hits do not match expected value!"
    assert len(unit_test_player.cache_new) == expected_cache_elements, "Cache elements do not match expected value!"
    assert len(unit_test_player.cache_old) == 0, "Cache elements do not match expected value!"


In [ ]:
test_minimax(
    unit_test_player,
    board,
    current_evaluation=1240,
    expected_evaluation=325,
    expected_move="d4c3",
    expected_nodes=2336,
    expected_cache_tries=2788,
    expected_cache_hits=557,
    expected_cache_elements=2231,
)

In [ ]:
# Test next move function (with memoized minimax result)
Exercise02AI_.test_next_move(
    unit_test_player,
    board,
    expected_move="d4c3",
    expected_nodes=0,
)